In [1]:
import time
import matplotlib.pyplot as plt
import pyRTC.utils as utils
import numpy as np

from pyRTC.hardware.ImakaDM import ImakaDM
from pyRTC.hardware import SpinnakerWFS

In [ ]:
conf = utils.read_yaml_file("config_felix.yaml")
dm_conf = conf["wfc"]
wfs_conf = conf["wfs"]

dm = ImakaDM(dm_conf)
wfs = SpinnakerWFS(wfs_conf)

In [ ]:
testcmd = np.zeros(dm.numActuators, dtype=np.float32)